In [ ]:
#take images from cam and mouse pos. look at the mouse to get where your looking for
import cv2
import pyautogui

cam = cv2.VideoCapture(0)
cv2.namedWindow("test")
img_counter = 0

while True:
    ret, frame = cam.read()
    if not ret:
        print("failed to grab frame")
        break
    cv2.imshow("test", frame)
    k = cv2.waitKey(1)
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        break
    elif k%256 == 32:
        # SPACE pressed
        x, y = pyautogui.position()
        #print(x,y)
        img_name = str(img_counter)+'-'+str(x)+'-'+str(y)
        img_name = "opencv_frame_{}.png".format(img_name)
        cv2.imwrite(img_name, frame)
        print("{} written!".format(img_name))
        img_counter += 1

cam.release()

cv2.destroyAllWindows()

In [1]:
# real time prediction

import numpy as np
import cv2
import matplotlib.pyplot as plt
import pyautogui
from tensorflow.keras.models import load_model
import time

loaded_model = load_model('./my_eyes.h5', compile = False)
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
eye_cascade = cv2.CascadeClassifier('haarcascade_eye.xml')

list_heatmap = []
start_time = time.time()
nb_boucle = 0

cap = cv2.VideoCapture(0)
stop = 0
while stop == 0:
    
    #img_crop1 = np.zeros(shape=(32, 32))
    #img_crop2 = np.zeros(shape=(32, 32))
    
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    #faces = face_cascade.detectMultiScale(img, scaleFactor = 1.1, minNeighbors = 5)
    
    ex,ey,ew,eh=1,1,1,1
    
    for (ex,ey,ew,eh) in faces:
        cv2.rectangle(img,(ex,ey),(ex+ew,ey+eh),(255,0,0),5)
        #print(ex,ey,eh,ew)
        milieu_crop_y = ey+eh/2
        milieu_crop_x = ex+ew/2
        distance_visage = 223/((eh+ew)/2)*0.4
        distance_visage = str(distance_visage)+'00000'
        distance_visage = float(distance_visage[0:5])
    
    image = img[ey:ey+eh, ex:ex+ew]
    eyes_array = []
    eyes = eye_cascade.detectMultiScale(image, scaleFactor = 1.1, minNeighbors = 5)
    
    for (eex,eey,eew,eeh) in eyes:
        if eey < image.shape[0]/2:
            cv2.rectangle(image,(eex,eey),(eex+eew,eey+eeh),(0,255,0),2) 
            #print(eex,eey,eeh,eew)
            eyes_array.append(eex)
            eyes_array.append(eey)
            eyes_array.append(eew)
            eyes_array.append(eeh)
            
    if len(eyes_array)== 8:        
        if eyes_array[0] > eyes_array[4]:  
            #right-left
            middle_left_x = ex+eyes_array[4]+eyes_array[6]/2
            middle_left_y = ey+eyes_array[5]+eyes_array[7]/2
            middle_right_x = ex+eyes_array[0]+eyes_array[2]/2
            middle_right_y = ey+eyes_array[1]+eyes_array[3]/2
            img_crop2 = image[eyes_array[1]:eyes_array[1]+eyes_array[3]+1, eyes_array[0]:eyes_array[0]+eyes_array[2]+1]
            img_crop1 = image[eyes_array[5]:eyes_array[5]+eyes_array[7]+1, eyes_array[4]:eyes_array[4]+eyes_array[6]+1]
        else :
            #left-right
            middle_right_x = ex+eyes_array[4]+eyes_array[6]/2
            middle_right_y = ey+eyes_array[5]+eyes_array[7]/2
            middle_left_x = ex+eyes_array[0]+eyes_array[1]/2
            middle_left_y = ey+eyes_array[2]+eyes_array[3]/2
            img_crop1 = image[eyes_array[1]:eyes_array[1]+eyes_array[3]+1, eyes_array[0]:eyes_array[0]+eyes_array[2]+1]
            img_crop2 = image[eyes_array[5]:eyes_array[5]+eyes_array[7]+1, eyes_array[4]:eyes_array[4]+eyes_array[6]+1]

        img_crop1 = img_crop1[2:img_crop1.shape[0]-2 , 2:img_crop1.shape[0]-2]
        img_crop2 = img_crop2[2:img_crop2.shape[0]-2 , 2:img_crop2.shape[0]-2]
        
        original_size_1 = img_crop1.shape[0]
        original_size_2 = img_crop2.shape[0]

        width = 32
        height = 32

        dim = (width, height) 
        img_crop1 = cv2.resize(img_crop1, dim, interpolation = cv2.INTER_AREA)
        img_crop2 = cv2.resize(img_crop2, dim, interpolation = cv2.INTER_AREA)
        
        img_crop1 = cv2.cvtColor(img_crop1, cv2.COLOR_BGR2GRAY)
        img_crop2 = cv2.cvtColor(img_crop2, cv2.COLOR_BGR2GRAY)
        
        img_2 = np.hstack((img_crop1, img_crop2))
        cv2.imshow('Eyes_frame', img_2) 
        
        img_crop1 = img_crop1/255 
        img_crop1 = img_crop1.reshape(1,32,32)
        #img_crop1 = np.expand_dims(img_crop1, axis=-1)
        #img_crop1 = img_crop1.reshape(1,32,32)
        #print(img_crop1.shape)
        
        img_crop11 = np.empty(0)
        img_crop11 = np.append(img_crop11, milieu_crop_x/640)
        img_crop11 = np.append(img_crop11, milieu_crop_y/480)
        img_crop11 = np.append(img_crop11, distance_visage)
        img_crop11 = np.append(img_crop11, middle_left_x/640)
        img_crop11 = np.append(img_crop11, middle_left_y/480)
        img_crop11 = np.append(img_crop11, original_size_1)
        img_crop11 = np.append(img_crop11, 0.5)
        img_crop11 = img_crop11.reshape(1, -1)
        #print(img_crop11.shape)
        pred1 = loaded_model.predict_on_batch([img_crop1,img_crop11])
        print(pred1)
        #nb1 = int(pred1[0 :pred1.find('-')])
        #nb2 = int(pred1[pred1.find('-')+1: len(pred1)])
        
        #img_crop2 = img_crop2.flatten()
        img_crop2 = img_crop2/255
        img_crop2 = img_crop2.reshape(1,32,32)
        
        img_crop22 = np.empty(0)
        img_crop22 = np.append(img_crop22, milieu_crop_x/640)
        img_crop22 = np.append(img_crop22, milieu_crop_y/480)
        img_crop22 = np.append(img_crop22, distance_visage)
        img_crop22 = np.append(img_crop22, middle_right_x/640)
        img_crop22 = np.append(img_crop22, middle_right_y/480)
        img_crop22 = np.append(img_crop22, original_size_2)
        img_crop22 = np.append(img_crop22, 0)
        img_crop22 = img_crop22.reshape(1, -1)
        #print(img_crop1.shape)
        pred2 = loaded_model.predict([img_crop2,img_crop22])
        print(pred2)
        #nb12 = int(pred2[0 :pred2.find('-')])
        #nb22 = int(pred2[pred1.find('-')+1: len(pred2)])
        
        x_final = ((pred1[0][0] + pred2[0][0])/2)*1920
        y_final = ((pred1[0][1]+ pred2[0][1])/2)*1080
        
        print(x_final,y_final)
        list_heatmap.append([x_final,y_final])
        
        pyautogui.moveTo(x_final, y_final) 
        nb_boucle = nb_boucle+1
    k = cv2.waitKey(30) & 0xff
    if k%256 == 27:
        # ESC pressed
        print("Escape hit, closing...")
        stop=1
        break     
        
end_time = time.time()
#print(list_heatmap)
print("durée :"+str(end_time-start_time))
print(nb_boucle)
cap.release()
cv2.destroyAllWindows()

[[0.31229988 0.16568048]]
[[0.655913  0.6895167]]
929.4843292236328 461.8064832687378
[[0.13166969 0.07970628]]
[[0.7598252  0.64158285]]
855.835132598877 389.4961452484131
[[0.7285025  0.44700217]]
[[0.6141068  0.43040767]]
1288.904914855957 473.8013005256653
[[0.31274098 0.06986399]]
[[0.7176214  0.59476906]]
989.1478729248047 358.9018392562866
[[0.52874917 0.10543186]]
[[0.78362155 0.73965913]]
1259.8759460449219 456.34913206100464
[[0.54634726 0.05067568]]
[[0.7723493  0.66195554]]
1265.9486389160156 384.82086181640625
[[0.08721103 0.10601693]]
[[0.75411534 0.7661288 ]]
807.6733016967773 470.95868468284607
[[0.31084773 0.11168531]]
[[0.7339574 0.7198496]]
1003.0129623413086 449.0288257598877
[[0.22309358 0.06503665]]
[[0.86702555 0.80934495]]
1046.5143585205078 472.1660649776459
[[0.632889  0.4892907]]
[[0.9219403  0.90115356]]
1492.6361846923828 750.8399105072021
[[0.6802463  0.17707525]]
[[0.8731526  0.83552504]]
1491.2628936767578 546.8041634559631
[[0.25972188 0.266379  ]]
[[0.

[[0.9764398  0.17700416]]
[[0.90519243 0.7060825 ]]
1806.3669204711914 476.86680793762207
[[0.7579155  0.19214198]]
[[0.9041822  0.78613514]]
1595.613784790039 528.2696270942688
[[0.8960514  0.10096474]]
[[0.90235865 0.6449445 ]]
1726.4736557006836 402.7909755706787
[[0.8852329 0.0714209]]
[[0.868734  0.5045058]]
1683.808250427246 311.00042939186096
[[0.9740373  0.08128233]]
[[0.7198754  0.20869868]]
1626.156234741211 156.58974409103394
[[0.9781877  0.21797314]]
[[0.9045737  0.73868036]]
1807.4509048461914 516.5928769111633
[[0.8299016  0.08447591]]
[[0.9119673 0.6474468]]
1672.1940994262695 395.2382826805115
[[0.9560001  0.16715033]]
[[0.87400764 0.4862933 ]]
1756.8074798583984 352.85956263542175
[[0.5571541  0.08471917]]
[[0.5752654  0.43824342]]
1087.122802734375 282.39978790283203
[[0.27933195 0.13089795]]
[[0.7925268  0.63033915]]
1028.9844131469727 411.06802582740784
[[0.36034057 0.11303559]]
[[0.73552245 0.5771776 ]]
1052.028465270996 372.7151298522949
[[0.2839372  0.11609843]]


[[0.10293598 0.09604369]]
[[0.1523984 0.5566729]]
245.12100219726562 352.46695160865784
[[0.16304785 0.13964207]]
[[0.26922753 0.6216674 ]]
414.9843692779541 411.1071002483368
[[0.54272705 0.09335099]]
[[0.79028004 0.69228566]]
1279.6868133544922 424.2437982559204
[[0.3350358  0.07247884]]
[[0.80553925 0.6665908 ]]
1094.952049255371 399.0976059436798
[[0.12501602 0.09051757]]
[[0.20684603 0.58893514]]
318.58755111694336 366.90446734428406
[[0.14272797 0.1219602 ]]
[[0.36075908 0.59440833]]
483.3475685119629 386.8390202522278
[[0.18383758 0.11636381]]
[[0.2727338  0.54431576]]
438.30854415893555 356.7669725418091
[[0.5508938  0.07966859]]
[[0.73920584 0.60014063]]
1238.4956359863281 367.0969748497009
[[0.27478456 0.14424153]]
[[0.52898663 0.5858966 ]]
771.6203498840332 394.2745864391327
[[0.08566673 0.05598208]]
[[0.6584565  0.49473464]]
714.3582916259766 297.38704919815063
[[0.89776057 0.3264663 ]]
[[1.1042843 0.8283894]]
1921.963119506836 623.6220932006836
[[0.9507709  0.34803328]]
[[

[[0.3464816  0.12182498]]
[[0.59595686 0.5837086 ]]
904.7409439086914 380.98812460899353
[[0.55163777 0.07600094]]
[[0.5031115  0.43170717]]
1012.5592803955078 274.1623914241791
[[0.54734296 0.06543111]]
[[0.5310199  0.44263563]]
1035.2284240722656 274.3560254573822
[[0.39419442 0.09861744]]
[[0.4999515  0.47330558]]
858.380126953125 308.83842945098877
[[0.36881196 0.10303351]]
[[0.41052395 0.33441016]]
748.1624794006348 236.21958374977112
[[0.26239654 0.08841294]]
[[0.4167429 0.419396 ]]
651.9738578796387 274.2168188095093
[[0.26252645 0.10548704]]
[[0.45226362 0.4329607 ]]
686.1985015869141 290.7617783546448
[[0.33792102 0.10811576]]
[[0.4862586  0.48374864]]
791.2124633789062 319.6067798137665
[[0.5165458  0.09438334]]
[[0.52780515 0.4440881 ]]
1002.5768280029297 290.7745885848999
[[0.61083144 0.03879717]]
[[0.55250865 0.31943688]]
1116.8064880371094 193.44638586044312
[[0.7280809  0.15315789]]
[[0.7511908 0.5660469]]
1420.1007843017578 388.37058305740356
[[0.7480305  0.10434424]]
[

[[0.2496755  0.22409563]]
585.3208351135254 189.7001177072525
[[0.3447794  0.18040976]]
[[0.39158505 0.31957543]]
706.9099044799805 269.9920016527176
[[0.35782275 0.06631473]]
[[0.6546634  0.46618226]]
971.9866561889648 287.5483739376068
[[0.6219864  0.11280575]]
[[0.64787024 0.39962927]]
1219.0624237060547 276.71491026878357
[[0.58791804 0.01573946]]
[[0.8100649 0.6782073]]
1342.0636367797852 374.731228351593
[[0.62640977 0.02573822]]
[[0.81414485 0.5822677 ]]
1382.9324340820312 328.3232080936432
[[0.8256828  0.17302182]]
[[0.8210123  0.56254697]]
1580.8273315429688 397.2071313858032
[[0.66491073 0.02450347]]
[[0.7462906  0.45500755]]
1354.753303527832 258.9359521865845
[[0.61920804 0.0148121 ]]
[[0.78642064 0.49456108]]
1349.4035339355469 275.06152153015137
[[0.75177294 0.05585254]]
[[0.7722943  0.45474055]]
1463.104476928711 275.72028279304504
[[0.87540483 0.18633145]]
[[0.8814637 0.6931138]]
1686.5938568115234 474.9004375934601
[[0.88254863 0.18119021]]
[[0.8698173 0.6009998]]
1682

[[0.5245797  0.23560773]]
[[0.5818988  0.70439804]]
1062.2193145751953 507.603120803833
[[0.09568184 0.0514413 ]]
[[0.10501059 0.35110363]]
192.66473293304443 217.37427055835724
[[0.12203359 0.06225767]]
[[0.17123753 0.63708967]]
281.54027938842773 377.6475512981415
[[0.12660365 0.15743123]]
[[0.36765516 0.6513529 ]]
474.488468170166 436.7434287071228
[[0.16251962 0.33507663]]
[[0.48873353 0.5110767 ]]
625.2030372619629 456.9227921962738
[[0.1592308  0.12336583]]
[[0.42990518 0.4933457 ]]
565.5705642700195 333.024240732193
[[0.8150583 0.8351   ]]
[[0.8599823 0.8579448]]
1608.0389785766602 914.2441821098328
[[0.87825453 0.8545877 ]]
[[0.9629971 0.902917 ]]
1767.601547241211 949.0525388717651
[[0.91880363 0.8793068 ]]
[[1.0991205 1.0215867]]
1937.2071075439453 1026.4824628829956
[[0.9144381 0.4152072]]
[[0.9817994 0.817698 ]]
1820.3881072998047 665.768780708313
[[0.9376276  0.29234806]]
[[1.0605721 0.3042951]]
1918.271713256836 322.18730092048645
[[0.94525194 0.07523719]]
[[1.1452007  0.